# A notebook to read a CSV template with Site/Individual/Skeleton Element info and load it to elab

Please enter file name in the following cell, and what is the delimiter in that file

In [1]:
CSVfilename="/Users/pierrespc/Documents/PostDocPasteur/aDNA/Import_eLAB/API_FUNCTIONALITIES/DownloadData/MasterTable/Outtables/2021-12-08.Master.eLab.Table.tsv"
delimiterFile="\t"

Please enter the one-line file where your token is saved in the following cell

In [2]:
tokenFile="/Users/pierrespc/Documents/PostDocPasteur/aDNA/Import_eLAB/API_FUNCTIONALITIES/credentials/tokenELAB"

Now preparing all required python libs

In [3]:
import os
import json
import requests
import csv
import pandas
import numpy
from apiclient import discovery, errors
from httplib2 import Http
from oauth2client import client, file, tools
import os.path

token = format(open(tokenFile,"r").readline().strip())
url = "https://elab-dev.pasteur.fr/api/v1/"
headers1 = {'Authorization': token, 'Accept': 'application/json','Content-Type':'application/json'}
headers2 = {'Authorization': token, 'Accept': 'application/json'}
params={}




Reading the data. SkelDict is to make sure thta if eLab configuration change we justy need to change here and not the excel template. 

In [4]:

DICT={"Site":{
        "Name":"Site",
        "Description":"None",
        "Note":"None",
        "Amount":"fixed_1",
        "Unit":"fixed_unit",
        "Main geographic region":"Site_Main geographic region",
        "Country":"Site_Country",
        "Province / Region":"Site_Province / Region",
        "Locality":"Site_Locality",
        "Latitude":"Site_LatChanged",
        "Longitude":"Site_LongChanged",
        "Site type":"Site_Site type",
        "Pictures":"None",
        "parentSampleID":"None"
    },

    "Individual":{
        "Name":"Individual",
        #"Description":"Individual_Archaeologist Observations",
        "Description":"None",
        "Note":"None",
        "Amount":"fixed_1",
        #"Unit":"fixed_Unit | pcs",
        "Unit":"fixed_unit",
        "parentSampleID":"Site",
        "Archaeologist ID":"Individual_Archaeologist ID",
        "Archaeologist group":"Individual_Archaeologist group",
        "Site Name":"Site",
        "Date":"Individual_Date",
        "Datation method":"Individual_Datation method",
        "Subsistence Strategy": "Individual_Subsistence Strategy",
        "Age":"Individual_Age",
        "Gender":"Individual_Gender",
        "Pictures":"None",
        "Linked individuals":"None"
    },

    "Skeleton Element":{"Name":"Skeleton Element",
         "From Individual":"Individual",
         "Description":"Skeleton Element_description",
         "Note":"None",
         "Amount":"fixed_1",
         #"Unit":"fixed_Unit | pcs",
         "Unit":"fixed_unit",
         "parentSampleID":"Individual",
         "Archaeologist sample ID":"Skeleton Element_Archaeologist sample ID",
         "Pictures Labelling":"Skeleton Element_Pictures Labelling",
         "Pictures Drilling":"None",
         "Bone type":"Skeleton Element_Bone type",
         "Skeleton element":"Skeleton Element_Skeleton element",
         "Exportation Permit Number":"Skeleton Element_Exportation Permit Number",
         "Observation Labelling":"Skeleton Element_Observation Labelling",
         "Observation Scanning":"Skeleton Element_Observation Scanning",
         "Observation Drilling":"None",
         "Scanned":"Skeleton Element_Scanned"

         
    }
}




Table=pandas.read_csv(CSVfilename,delimiter=delimiterFile)

Table

,Unnamed: 0,Site,Site_Pictures,Site_Main geographic region,Site_Country,Site_Province / Region,Site_Locality,Site_Latitude,Site_Longitude,Site_Site type,...,Indexed Library_QC-1 Elution Volume of Library (uL),Indexed Library_QC-1 Average Read Length (bp),Indexed Library_Library Profile,Indexed Library_QC-1 date,Indexed Library_Volume taken for indexing (uL),Indexed Library_Indexing PCR date,Indexed Library_Sample well in library profile file,Indexed Library_description,Indexed Library_Quantity,Indexed Library_note
0,0,Alto Verde-Gruta 11,NaN,San Juan,Argentina,San Juan,NaN,-31.35,-69.44,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Alto Verde-Gruta 11,NaN,San Juan,Argentina,San Juan,NaN,-31.35,-69.44,-,...,40.0,238.0,NaN,2021-07-23,20.0,2021-07-20,H6,NaN,3.5e-05Liter,NaN
2,2,Alto Verde-Gruta 11,NaN,San Juan,Argentina,San Juan,NaN,-31.35,-69.44,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Alto Verde-Gruta 11,NaN,San Juan,Argentina,San Juan,NaN,-31.35,-69.44,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Alto Verde-Gruta 11,NaN,San Juan,Argentina,San Juan,NaN,-31.35,-69.44,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1081,1081,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1082,1082,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,40.0,148.0,NaN,2021-07-15,20.0,2021-07-12,H10,NaN,3.5e-05Liter,NaN
1083,1083,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,40.0,126.0,NaN,2021-07-23,20.0,2021-07-20,H7,NaN,3.5e-05Liter,NaN
1084,1084,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,40.0,154.0,NaN,2021-07-23,20.0,2021-07-20,D10,NaN,3.5e-05Liter,NaN


Format longitud and latitude 

In [5]:
Table.loc[Table['Site_Latitude'].isnull(),"Site_Latitude"]=None
Table.loc[Table['Site_Longitude'].isnull(),"Site_Longitud"]=None
Table.loc[numpy.isnan(Table['Site_Latitude']),"Site_Latitude"]=None
Table.loc[numpy.isnan(Table['Site_Longitude']),"Site_Longitud"]=None


Table.loc[Table['Site_Latitude'].isin([ "Undefined","desconocido","nan","NA"]),"Site_Latitude"]=None
Table.loc[Table['Site_Longitude'].isin([ "Undefined","desconocido","nan","NA"]),"Site_Longitud"]=None


###change coordinates
def changeCOORfunction(coord):
    stuf="º"
    stuf2="°"
    if coord is None or numpy.isnan(coord):
        #return([None,None,None,None])
        return(None)
                      
    coord=format(coord)
    coordFed=coord
    #print(coord)
    #####HERE CHANGE ALL WEIRD CHARACTERS THAT CAN COME ON THE WAY
    coord=coord.replace("39° 06 ́","39°06'")
    coord=coord.replace("63° 47 ́","63°47'") 
    ##change weird degrees characters
    coord=coord.replace(stuf2,stuf)
    ##change weird minutes characters
    coord=coord.replace("``","\"")
    coord=coord.replace("´´","\"")
    coord=coord.replace("”","\"")
    coord=coord.replace("''","\"")
    coord=coord.replace("’’","\"")
    coord=coord.replace("“","\"")
    ##change weird seconds characters
    coord=coord.replace(" ´","'")
    coord=coord.replace("`","'")
    coord=coord.replace("´","'")
    coord=coord.replace("’","'")
    ###we now that it is all South and West
    coord=coord.replace(" ","")
    coord=coord.replace("S","")
    coord=coord.replace("O","")
    coord=coord.replace("W","")
    ###change decimal character
    coord=coord.replace(",",".")

    ##read degrees
    if len(coord.split(stuf)) ==2 :
        if coord.split(stuf)[1] == "":
            if stuf not in coord:
                coord=coord + stuf + "0'"
            else:
                coord=coord+"0'"
    elif len(coord.split(stuf)) == 1 :
        if stuf not in coord:
            coord=coord + stuf + "0'"
        else:
            coord=coord+"0'"
    else:
        print("splitting minute/second " + coordFed + "-->" + coord)
            
    deg=coord.split(stuf)[0]
    ###read minutes and seconds
    tmp=coord.split(stuf)[1]
    minute=tmp.split("'")[0]
    if len(tmp.split("'")) != 2:
        print( coord.split(stuf))
        print("splitting minute/second " + coordFed + "-->" + coord)
        raise()
    else:
        if tmp.split("'")[1] == "":
            sec=0
        else:
            sec=tmp.split("'")[1]
            sec=sec.replace("\"","")

    #print([coordFed,coord,deg,minute,sec])
    ####verify all read ok!
    if numpy.isnan(float(deg)):
        print("pb numerical degree" + coordFed + "-->" +coord + " (" + deg + ")")
        raise()
    if numpy.isnan(float(minute)):
        print(minute)
        print("pb numerical minute" + coordFed + "-->" +coord + " (" + minute + ")")
        raise()
    if numpy.isnan(float(sec)):
        print(sec)
        print("pb numerical sec" + coordFed + "-->" + coord + " (" + sec + ")")
        raise()
  
    deg=float(deg)
    minute=float(minute)
    sec=float(sec)
    new=deg+minute/60+sec/3600
    if(new<0):
        return(new)
    else:
        return(-new)



#print(rawTab.loc[rawTab['Latitude']==""])
Table['Site_LatChanged']=Table['Site_Latitude'].apply(changeCOORfunction)
Table['Site_LongChanged']=Table['Site_Longitude'].apply(changeCOORfunction)

Table[["Site_LatChanged","Site_Latitude","Site_LongChanged","Site_Longitude"]]

,Site_LatChanged,Site_Latitude,Site_LongChanged,Site_Longitude
0,-31.35,-31.35,-69.44,-69.44
1,-31.35,-31.35,-69.44,-69.44
2,-31.35,-31.35,-69.44,-69.44
3,-31.35,-31.35,-69.44,-69.44
4,-31.35,-31.35,-69.44,-69.44
...,...,...,...,...
1081,NaN,NaN,NaN,NaN
1082,NaN,NaN,NaN,NaN
1083,NaN,NaN,NaN,NaN
1084,NaN,NaN,NaN,NaN


Prepare all the eLab-API keys necessary to down and upload data

In [6]:
def BadRequest(myReq,code=200):
    return(myReq.status_code !=code)


r = requests.get(url + "sampleTypes", headers = headers2)
if BadRequest(r,200):
    r.raise_for_status()
data = r.json()
types = {}
for typ in data.get("data"):
    types[format(typ.get("name"))] = format(typ.get("sampleTypeID"))

print(types)


FeateLab={}
for sampTY in ["Individual","Skeleton Element","Site"]:
    r = requests.get(url + "sampleTypes/" + types[sampTY] + "/meta", headers = headers2)
    if BadRequest(r,200):
        r.raise_for_status()
    data = r.json()
    FeateLab[sampTY] = {}
    for feat in ['Name','Description','Note','Amount','Unit',"parentSampleID"]:
        FeateLab[sampTY][feat] = {"ID": "notMeta"}
    for feat in data.get("data"):
        FeateLab[sampTY][format(feat.get("key"))] = { "ID":format(feat.get("sampleTypeMetaID")),
                                              "TYPE":format(feat.get("sampleDataType"))}

print(FeateLab)

{'Individual': '39466', 'Site': '39468', 'Skeleton Element': '39469', 'Extract': '39470', 'Indexed Library': '39494', 'Library pool': '39495', 'Non Indexed Library': '39556', 'Bone pellet': '39599'}
{'Individual': {'Name': {'ID': 'notMeta'}, 'Description': {'ID': 'notMeta'}, 'Note': {'ID': 'notMeta'}, 'Amount': {'ID': 'notMeta'}, 'Unit': {'ID': 'notMeta'}, 'parentSampleID': {'ID': 'notMeta'}, 'Archaeologist group': {'ID': '244121', 'TYPE': 'CHECKBOX'}, 'Archaeologist ID': {'ID': '244132', 'TYPE': 'TEXT'}, 'Site Name': {'ID': '244133', 'TYPE': 'SAMPLELINK'}, 'Date': {'ID': '244134', 'TYPE': 'TEXT'}, 'Datation method': {'ID': '244135', 'TYPE': 'TEXTAREA'}, 'Age': {'ID': '244136', 'TYPE': 'TEXT'}, 'Gender': {'ID': '244137', 'TYPE': 'COMBO'}, 'Linked individuals': {'ID': '244138', 'TYPE': 'TEXT'}, 'Pictures': {'ID': '244141', 'TYPE': 'FILE'}}, 'Skeleton Element': {'Name': {'ID': 'notMeta'}, 'Description': {'ID': 'notMeta'}, 'Note': {'ID': 'notMeta'}, 'Amount': {'ID': 'notMeta'}, 'Unit': {'

Check the columns in  Table are recognized here. If no lines in output, you are just fine.

In [7]:
for sampTY in  FeateLab.keys():
    for feat in FeateLab[sampTY].keys():
        if feat not in DICT[sampTY].keys():
            exit(sampTY+": "+feat + "--> NOT IN DICTIONARY")
        
    for feat in DICT[sampTY].keys():
        if feat not in FeateLab[sampTY].keys():
            exit(sampTY+": "+feat + "--> NOT IN eLAB")
        


We get all the possible values for checkboxes and dropdown features of Extracts and check our extractTable table is fine. If no lines in output, you're just fine

In [8]:
for sampTY in FeateLab.keys():
    r = requests.get(url + "sampleTypes/" + types[sampTY] + "/meta", headers = headers2)
    data = r.json()
    for feat in data.get("data"):
        if feat.get("sampleDataType") == "CHECKBOX" or feat.get("sampleDataType") == "COMBO":
            OptionELAB=feat.get("optionValues")
            key=feat.get("key")
            if DICT[sampTY][key].startswith("fixed"):
                tabVal=DICT[sampTY][key].split("_")[1]
                if tabVal not in OptionELAB:
                    exit(sampTY+": " + tabVal + "-- not mapped in eLab for " + key)
            else:
                Table.loc[Table[DICT[sampTY][key]].isnull(),DICT[sampTY][key]]="NA"
                for tabVal in Table[DICT[sampTY][key]].unique():
                    if tabVal not in OptionELAB:
                        exit(sampTY+": " + tabVal + "-- not mapped in eLab for " + key)



Now, we make the json for each extract and we upload or update in eLab!
Change the default prompt line:
- put y if you are sure you want to overwrite already loaded info in eLab, 
- put n if you are sure you want to leave already loaded info in eLab (although it doesn't match info in your table)
- put anything else if you want a case by case prompt

In [9]:


####get all registered skeleton element and extracts
registered = {}
for name in ["Skeleton Element","Individual","Site"]:
    #print(name)
    r = requests.get(url + "samples" , headers = headers2, params = {'sampleTypeID': types[name]})
    if BadRequest(r,200):
        r.raise_for_status()
    data = r.json()
    myList = {}
    for sam in data.get("data"):
        if format(sam.get("name")) in myList.keys():
            print(name + ": " + sam.get("name") + " duplicated")
            break
        myList[format(sam.get("name"))]=format(sam.get("sampleID"))
    registered[name] = myList



In [ ]:
defaultPrompt="?"

for sampTy in ["Site","Individual","Skeleton Element"]:
#for sampTy in ["Skeleton Element"]:
    
    ###make an unique TableTYPE
    TableTYPE={}
    for fea in DICT[sampTy].values():
        if fea != "None" and not fea.startswith("fixed"):
            TableTYPE[fea]=Table[fea]
        
    TableTYPE=pandas.DataFrame(TableTYPE).drop_duplicates()
    
    ###iterate over extracts in table
    for index,name in TableTYPE[DICT[sampTy]['Name']].items():
        print(name)
        if format(name)=="nan":
            continue
        patch=False
        ###get what is already uploaded fior that extract in eLab
        if name in registered[sampTy].keys():
            patch=True
            id=registered[sampTy][name]
            r=requests.get(url + "samples/"+id, headers = headers2)
            if BadRequest(r,200):
                r.raise_for_status()
            r=r.json()
        
            ###change to lower case all the keys because API sometimes use upper, lower for different request (A MESS!)
            dataLoaded={}
            for oldkey in r:
                newkey=oldkey.lower()
                dataLoaded[newkey] = r[oldkey]
        ####prepare the Data to be loaded
        Data={}
        for fea in FeateLab[sampTy].keys():
            if FeateLab[sampTy][fea]['ID'] == "notMeta" and fea not in ["Amount","Unit"]:
                ###fixed value (from dico)
                if DICT[sampTy][fea].startswith("fixed"):
                    element=DICT[sampTy][fea].split("_")[1]
                ###WHEN NO VALUE to enter
                elif DICT[sampTy][fea]=="None":
                    if fea == "parentSampleID":
                        element=0
                    else:
                        element="NA"
                elif fea == "parentSampleID":
                    if sampTy == "Site":
                        continue
                    element=registered[DICT[sampTy]["parentSampleID"]][TableTYPE[DICT[sampTy]["parentSampleID"]][index]]
                else:
                    element=TableTYPE[DICT[sampTy][fea]][index]
                ###check delta when patching
                if patch:
                    elementLoaded=dataLoaded[fea.lower()]
                    if format(elementLoaded) != format(element):
                        print("For "+sampTy+" "+name+", do you want to update the "+fea+" field? That is: "+format(element)+ " vs what already loaded: "+format(elementLoaded))
                        if DICT[sampTy][fea]=="None":
                            print("no update for None features")
                            prompt="n"
                        else:
                            prompt=defaultPrompt
                            while prompt not in ["y","n"]:
                                prompt = input("replace y/n??")
                        if prompt == "n":
                             element=elementLoaded
                Data[fea]=element
        ###case of updating
        if patch:
            DR=requests.patch(url + "samples/"+id, headers = headers2,data = Data)
            if BadRequest(DR,204):
                DR.raise_for_status()
        else:
            ###case of uploading
            #print(name + "uploading")
            patch=False
            Data["sampleTypeID"]=types[sampTy]
            Data["Name"]=name
            DR=requests.post(url + "samples/", headers = headers2,data = Data)   

        ####check the Data loading was correct
        if BadRequest(DR,204):
            DR.raise_for_status()
        ###actualize the registered[<sample type>] list (checking we did not duplicated anything here)
        r=requests.get(url + "samples/forNames?names="+name, headers = headers2)
        if BadRequest(r,200):
            r.raise_for_status()
        data=r.json()
        sam=data.get("data")
        if len(sam)!=1:
            exit("different "+sampTy+" entries (" + str(len(sam)) + ") for name "+name)    
        else:
            sam=sam[0]
            id=str(sam.get("sampleID"))
            #print("Data OK for "+ name + " (" + id + ")")
            registered[sampTy][name]=id

        ###patch the metaData
        if patch:
            #print("patching meta so need to heck if differences for "+name)
            MDR=requests.get(url + "samples/"+id+"/meta", headers = headers2)
            if BadRequest(MDR,200):
                MDR.raise_for_status()
            data=MDR.json().get("data")
            metaLoaded={}
            for i in data:
                metaLoaded[i["key"]]=str(i["value"])

        for fea in FeateLab[sampTy].keys():
            needToPatch=False
            MDR=requests.get(url + "samples/"+id+"/meta", headers = headers2)
            if BadRequest(MDR,200):
                MDR.raise_for_status()
            ###get new element to be loaded
            if FeateLab[sampTy][fea]['ID'] != "notMeta" and FeateLab[sampTy][fea]['TYPE'] != "FILE":
                ###fixed value (from dico)
                if DICT[sampTy][fea].startswith("fixed"):
                    element=DICT[sampTy][fea].split("_")[1]
                    MetaData={"key": fea,
                              "sampleTypeMetaID": int(FeateLab[sampTy][fea]['ID']),
                              "value": element,
                              "sampleDataType": FeateLab[sampTy][fea]['TYPE']}
                elif DICT[sampTy][fea]=="None":
                    element="NA"
                    MetaData={"key": fea,
                              "sampleTypeMetaID": int(FeateLab[sampTy][fea]['ID']),
                              "value": element,
                              "sampleDataType": FeateLab[sampTy][fea]['TYPE']}
                elif (fea == "From Individual" and sampTy =="Skeleton Element") or (fea == "Site Name" and sampTy =="Individual"):
                    sisi=TableTYPE[DICT[sampTy][fea]][index]
                    IDsisi=registered[DICT[sampTy][fea]][sisi]
                    element=sisi+"|"+IDsisi
                    samples={"sampleID": IDsisi,"name": sisi}
                
                    MetaData={
                        "sampleTypeMetaID": int(FeateLab[sampTy][fea]['ID']),
                        "sampleDataType": FeateLab[sampTy][fea]['TYPE'],
                        "samples": samples,
                        "key": fea,
                        "value": element
                    }
                else:
                    element=TableTYPE[DICT[sampTy][fea]][index]
                    if format(element)=="nan" or format(element)=="" or format(element)==" ":
                        element="NA"
                    MetaData={"key": fea,
                              "sampleTypeMetaID": int(FeateLab[sampTy][fea]['ID']),
                              "value": element,
                              "sampleDataType": FeateLab[sampTy][fea]['TYPE']}
            
                ###check if this is a new entry or not
                if patch:
                    ###check if new element is similar to what already loaded
                    if fea not in metaLoaded.keys(): 
                        needToPatch=True
                    elif metaLoaded[fea] != str(element):
                        print("difference for " + name + "(feature: " + fea + ") " + str(element) + " vs loaded : " + metaLoaded[fea])
                        if DICT[sampTy][fea]=="None":
                            print("no update for None features")
                            prompt="n"
                        else:
                            prompt=defaultPrompt
                            while prompt not in ["y","n"]:
                                prompt = input("???replace y/n??")
                        if prompt == "y":
                            needToPatch=True
                else:
                    needToPatch=True
        
                if needToPatch:
                    #print(MetaData)      
                    MDR=requests.put(url + "samples/"+id+"/meta", headers = headers2,data = MetaData)
                    ####check the MetaData loading was correct
                    if BadRequest(MDR,204):
                        MDR.raise_for_status()
        #print("metadata OK for "+ name + " (" + id + ")")
        ###patch the quantity
        Quant={}
        Note=None
        for fea in ['Amount','Unit']:
            if DICT[sampTy][fea].startswith("fixed"):
                element=DICT[sampTy][fea].split("_")[1]
            elif DICT[sampTy][fea]=="None":
                element="NA"
            else:
                element=TableTYPE[DICT[sampTy][fea]][index]
                if format(element)=="nan":
                    element=0
                elif "<" in format(element):
                    Note="actual amount reported: "+element
                    element=0
            Quant[fea]=element
        Quant["displayUnit"]=Quant["Unit"].capitalize()
        Quant["fullAmount"]=Quant["Amount"]
        QR=requests.put(url + "samples/" + id + "/quantity", headers = headers2, data = Quant)
        if BadRequest(QR,204):
            QR.raise_for_status()
            ###put actual weight in note when there is a "<"

        if Note is not None:
            r=requests.get(url + "samples/"+id, headers = headers2)
            if BadRequest(r,200):
                r.raise_for_status()
            Data=r.json()
            Data["note"]=Data["note"]+" / "+ Note
            r=requests.patch(url + "samples/"+id, headers = headers2,data = Data)
            if BadRequest(r,204):
                r.raise_for_status()
print("finished")




## Assignation to storage
first organize the storage IDs (a bit messy but eLab treats all storage levels similarly for sample assignation)

In [ ]:
storageByID={}
r=requests.get(url+"/storageLayers",headers=headers1)
if BadRequest(r,200):
    r.raise_for_status()
    
stoData=r.json().get("data")
for sto in stoData:
    
    storageByID[sto["storageLayerID"]]={"name":sto["name"],"parentID":sto["parentStorageLayerID"]}

def getParentSto(ID,stoDict):
    if stoDict[ID]["parentID"]==0:
        return(stoDict[ID]["name"])
    else:
        return(getParentSto(stoDict[ID]["parentID"],stoDict)+", "+stoDict[ID]["name"])
    
storage={}
storageRev={}
for stoID in storageByID.keys():
    name=getParentSto(stoID,storageByID)
    storage[name]=stoID
    storageRev[stoID]=name
    


### Individual and Site  assignation to Artefactual Storage 


In [ ]:
for sampTy in [ "Skeleton Element", "Individual","Site"]:
    TableTYPE={}
    for fea in DICT[sampTy].values():
        if fea != "None" and not fea.startswith("fixed"):
            TableTYPE[fea]=Table[fea]
        
    TableTYPE=pandas.DataFrame(TableTYPE).drop_duplicates()
    if sampTy+"_Storage" not in TableTYPE.keys():
        TableTYPE[sampTy+"_Storage"]="" 
    if len(TableTYPE.loc[TableTYPE[sampTy+"_Storage"] == "",sampTy+"_Storage"])>0:
        UnspecifiedStorage="?"
        while UnspecifiedStorage not in storage:
            UnspecifiedStorage=input("Storage for "+sampTy+" with missing storage info in input table?")
        TableTYPE.loc[TableTYPE[sampTy+"_Storage"] =="",sampTy+"_Storage"]=UnspecifiedStorage

    for index,name in TableTYPE[DICT[sampTy]['Name']].items():
        if format(name)=="nan":
            continue
        id=format(registered[sampTy][name])
        r=requests.get(url+"/samples/"+id,headers=headers1,data={})
        if BadRequest(r,200):
            r.raise_for_status()
        IDstoAlready=r.json()['storageLayerID']
        IDsto=storage[ TableTYPE[sampTy+"_Storage"][index]]
        if IDstoAlready != 0:
            if(format(IDstoAlready) != format(IDsto)):
                prompt=defaultPrompt
                #if storageRev[IDstoAlready] in ["In Copenhagen","Nico office","In Tartu","Unknown"]:
                #    prompt="n"
                #elif storageRev[IDstoAlready] == "With Lumila":
                #    prompt="y"
                while prompt not in ["y","n"]:
                    prompt=input("move "+name+" from "+storageRev[IDstoAlready]+ " to "+storageRev[IDsto]+" (y/n)?")
            if prompt == "y":
                r=requests.post(url+"/samples/moveToLayer/"+format(IDsto)+"?sampleIDs="+id,headers=headers1,data={})
                if BadRequest(r,204):
                    r.raise_for_status()
        

## Assign individual and Skeleton Element to Labelling experiment

In [ ]:
r = requests.get(url + "experiments", headers = headers2,params = params)
data = r.json()
experiments = {}
for exp in data.get("data"):
    experiments[format(exp.get("name"))] = format(exp.get("experimentID"))

for expe in ["Labelling process"]:
    #print(expe)
    idExpe=experiments[expe]
    r=requests.get("https://elab-dev.pasteur.fr/api/v1/experiments/"+idExpe+"/sections",headers=headers1)
    if r.status_code != 200:
        print(r.status_code)
        print(r.raise_for_status())
    if r.json().get("recordCount") == 0:
        print("no record")
        continue
    SampleIN={}
    SampleOUT={}
    for data in r.json().get("data"):
        if data["sectionType"] == "SAMPLESIN":
            SampleIN[data["sectionHeader"]]=data["expJournalID"]
        elif data["sectionType"] == "SAMPLESOUT":
            SampleOUT[data["sectionHeader"]]=data["expJournalID"]
    experiments[expe]={"ID":idExpe,
                      "sampleIN":SampleIN,
                      "sampleOUT":SampleOUT}




sampleOUT=experiments["Labelling process"]["sampleOUT"]["Labelled Skeleton elements "]
sampleIN=experiments["Labelling process"]["sampleIN"]["Individuals labelled"]



In [ ]:
listOUT=[]
listIN=[]
for inName in registered["Skeleton Element"].keys():
    inID=registered["Skeleton Element"][inName]
    listOUT.append(inID)
    ###get the parent individual
    r=requests.get(url+"/samples/"+inID+"/parent",headers=headers1)
    if BadRequest(r,200):
        r.raise_for_status()
        break
    rjson=r.json()
    outName=rjson.get("name")
    listIN.append(rjson.get("sampleID"))



In [ ]:
r=requests.put(url+"/experiments/sections/"+format(sampleOUT)+"/samples",headers=headers1,data = format(listOUT))
if BadRequest(r,204):
    r.raise_for_status()



In [ ]:
r=requests.put(url+"/experiments/sections/"+format(sampleIN)+"/samples",headers=headers1,data = format(listIN))
if BadRequest(r,204):
    r.raise_for_status()